In [1]:
import pyranges as pr

# Human (GRCh37.p13, GENCODE Release 39)

In [46]:
GRCh37 = pr.read_gtf('GTF/gencode.v39lift37.annotation.gtf')

## Human Pseudogene
14,256 pgenes (48,220 exons) -> 8,424 pgenes (34,420 exons) after removing exons overlapping with protein coding genes and lncRNAs -> 4,279 pgenes (14,207 exons) after selecting exons within unique genomic regions -> 4,067 pgenes (10, 415 exons) after removing exons shorter than 100 bp.

In [47]:
## extract pgenes from the GTF file
pgene_type = ['processed_pseudogene', 'transcribed_processed_pseudogene', 'transcribed_unitary_pseudogene', 'transcribed_unprocessed_pseudogene',
              'translated_processed_pseudogene', 'translated_unprocessed_pseudogene', 'unitary_pseudogene', 'unprocessed_pseudogene']

In [48]:
pgene = GRCh37[(GRCh37.Feature == 'gene') & ([j in pgene_type for j in GRCh37.gene_type])]
pexon = GRCh37[(GRCh37.Feature == 'exon') & ([j in pgene_type for j in GRCh37.gene_type])]
protein_coding = GRCh37[(GRCh37.Feature == 'gene') & (GRCh37.gene_type == 'protein_coding')]
protein_coding_exon = GRCh37[(GRCh37.Feature == 'exon') & (GRCh37.gene_type == 'protein_coding')]
lncRNA = GRCh37[(GRCh37.Feature == 'gene') & (GRCh37.gene_type == 'lncRNA')]

In [49]:
print(len(set(pexon.gene_id)), len(pexon))

14256 48220


In [50]:
overlap_1 = pexon.count_overlaps(protein_coding, strandedness = False, overlap_col = "Count")
overlap_2 = pexon.count_overlaps(lncRNA, strandedness = False, overlap_col = "Count")
pexon = pexon[(overlap_1.Count == 0) & (overlap_2.Count == 0)]

In [51]:
print(len(set(pexon.gene_id)), len(pexon))

8424 34420


In [52]:
pexon.to_gtf('pgene/human/pexon_filtered_by_overlappings.gtf')

In [53]:
! gtf2bed < pgene/human/pexon_filtered_by_overlappings.gtf > pgene/human/pexon_filtered_by_overlappings.bed

In [55]:
! cat mappability/human/hg19.bedgraph | awk '{if($4>0.95) print $0}' | \
bedtools coverage -a pgene/human/pexon_filtered_by_overlappings.bed -b - | \
awk -F '\t' '{if($NF==1) print $0}' | awk -F '\t' 'NF-=4' > pgene/human/pexon_filtered_by_mappability.bed 

In [56]:
! awk '{print $1"\t"$7"\t"$8"\t"($2+1)"\t"$3"\t"$5"\t"$6"\t"$9"\t"(substr($0, index($0,$10)))}' pgene/human/pexon_filtered_by_mappability.bed > pgene/human/pexon_filtered_by_mappability.gtf
! sed -i '' 's/zero_length_insertion.*//g' pgene/human/pexon_filtered_by_mappability.gtf

In [57]:
pexon = pr.read_gtf('pgene/human/pexon_filtered_by_mappability.gtf')
print(len(set(pexon.gene_id)), len(pexon))

4279 14207


In [58]:
pexon = pexon[pexon.lengths() >= 100]
print(len(set(pexon.gene_id)), len(pexon))

4067 10415


In [59]:
pexon.drop_duplicate_positions()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_name,tag,exon_number,exon_id,remap_original_location,transcript_support_level,havana_transcript,ont,hgnc_id
0,chr1,HAVANA,exon,995108,995226,.,+,.,ENSG00000217801.11_10,transcribed_unprocessed_pseudogene,...,processed_transcript,ENST00000427998,TAGENE,1,ENSE00003923178.1_1,chr1:+:1059729-1059846,1,OTTHUMT00000099676.1_3,NaN,NaN
1,chr1,HAVANA,exon,995113,995226,.,+,.,ENSG00000217801.11_10,transcribed_unprocessed_pseudogene,...,processed_transcript,ENST00000692266,TAGENE,1,ENSE00001669701.1_1,chr1:+:1059734-1059846,NaN,NaN,NaN,NaN
2,chr1,HAVANA,exon,995656,995773,.,+,.,ENSG00000217801.11_10,transcribed_unprocessed_pseudogene,...,processed_transcript,ENST00000394517,RNA_Seq_supported_partial,2,ENSE00003841587.1_1,chr1:+:1060277-1060393,5,OTTHUMT00000001276.2_3,NaN,NaN
3,chr1,HAVANA,exon,995659,995773,.,+,.,ENSG00000217801.11_10,transcribed_unprocessed_pseudogene,...,processed_transcript,ENST00000427998,TAGENE,2,ENSE00001702890.1_1,chr1:+:1060280-1060393,1,OTTHUMT00000099676.1_3,NaN,NaN
4,chr1,HAVANA,exon,999585,999781,.,+,.,ENSG00000217801.11_10,transcribed_unprocessed_pseudogene,...,processed_transcript,ENST00000688131,TAGENE,4,ENSE00003923512.1_1,chr1:+:1064206-1064401,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8264,chrY,HAVANA,exon,23839131,23839321,.,-,.,ENSG00000229416.1_7,unprocessed_pseudogene,...,unprocessed_pseudogene,USP9YP8-201,Ensembl_canonical,3,ENSE00001774789.1_1,chrY:-:21677246-21677435,NA,OTTHUMT00000101918.1_4,PGO:0000005,HGNC:38424
8265,chrY,HAVANA,exon,23963725,23963902,.,-,.,ENSG00000277519.1_7,unprocessed_pseudogene,...,unprocessed_pseudogene,OFD1P16Y-201,Ensembl_canonical,2,ENSE00003745229.1_1,chrY:-:21817579-21817755,NA,OTTHUMT00000475886.1_4,PGO:0000005,HGNC:18213
8266,chrY,HAVANA,exon,23964224,23964365,.,-,.,ENSG00000277519.1_7,unprocessed_pseudogene,...,unprocessed_pseudogene,OFD1P16Y-201,Ensembl_canonical,1,ENSE00003732110.1_1,chrY:-:21818078-21818218,NA,OTTHUMT00000475886.1_4,PGO:0000005,HGNC:18213
8267,chrY,HAVANA,exon,24453706,24454098,.,-,.,ENSG00000229940.1_5,unprocessed_pseudogene,...,unprocessed_pseudogene,TSPY22P-201,Ensembl_canonical,1,ENSE00001663391.1_1,chrY:-:22307560-22307951,NA,OTTHUMT00000101928.1_4,PGO:0000005,HGNC:38522


In [60]:
pexon.to_gtf('pgene/human/pexon_human.gtf')

In [3]:
import pyranges as pr
pexon = pr.read_gtf('GTF/pgene/human/pexon_human.gtf')

In [8]:
len(set(pexon.gene_id))

4067

In [20]:
from collections import Counter
Counter(pexon.df[['gene_id','gene_type']].drop_duplicates()['gene_type'])

Counter({'transcribed_unprocessed_pseudogene': 310,
         'processed_pseudogene': 2753,
         'unprocessed_pseudogene': 735,
         'unitary_pseudogene': 47,
         'transcribed_processed_pseudogene': 116,
         'transcribed_unitary_pseudogene': 106})

## Human Protein-coding Gene
20,111 protein-coding genes (1,303,925 exons) -> 19,198 protein-coding genes (1,196,225 exons) after removing exons overlapping with pseudogenes and lncRNAs -> 17,880 protein-coding genes (1,094,254 exons) after selecting exons within unique genomic regions -> 17,599 protein-coding genes (720,075 exons) after removing exons shorter than 100 bp.

In [71]:
print(len(set(protein_coding_exon.gene_id)), len(protein_coding_exon))

20111 1303925


In [72]:
overlap_1 = protein_coding_exon.count_overlaps(pgene, strandedness = False, overlap_col = "Count")
overlap_2 = protein_coding_exon.count_overlaps(lncRNA, strandedness = False, overlap_col = "Count")
protein_coding_exon = protein_coding_exon[(overlap_1.Count == 0) & (overlap_2.Count == 0)]

In [73]:
print(len(set(protein_coding_exon.gene_id)), len(protein_coding_exon))

19198 1196225


In [42]:
protein_coding_exon.to_gtf('protein_coding/human/protein_coding_exon_filtered_by_overlappings.gtf')

In [43]:
! gtf2bed < protein_coding/human/protein_coding_exon_filtered_by_overlappings.gtf > protein_coding/human/protein_coding_exon_filtered_by_overlappings.bed

In [44]:
! cat mappability/human/hg19.bedgraph | awk '{if($4>0.95) print $0}' | \
bedtools coverage -a protein_coding/human/protein_coding_exon_filtered_by_overlappings.bed -b - | \
awk -F '\t' '{if($NF==1) print $0}' | awk -F '\t' 'NF-=4' > protein_coding/human/protein_coding_exon_filtered_by_mappability.bed 

In [45]:
! awk '{print $1"\t"$7"\t"$8"\t"($2+1)"\t"$3"\t"$5"\t"$6"\t"$9"\t"(substr($0, index($0,$10)))}' protein_coding/human/protein_coding_exon_filtered_by_mappability.bed  > protein_coding/human/protein_coding_exon_filtered_by_mappability.gtf
! sed -i '' 's/zero_length_insertion.*//g' protein_coding/human/protein_coding_exon_filtered_by_mappability.gtf

In [75]:
protein_coding_exon = pr.read_gtf('protein_coding/human/protein_coding_exon_filtered_by_mappability.gtf')
print(len(set(protein_coding_exon.gene_id)), len(protein_coding_exon))

17880 1094254


In [76]:
protein_coding_exon = protein_coding_exon[protein_coding_exon.lengths() >= 100]
print(len(set(protein_coding_exon.gene_id)), len(protein_coding_exon))

17599 720075


In [78]:
protein_coding_exon.to_gtf('protein_coding/human/protein_coding_human.gtf')

### Extract ALL Coordinates of Pseudogenes and Protein-coding Genes

In [4]:
processed_pgene_type = ['processed_pseudogene', 'transcribed_processed_pseudogene', 'translated_processed_pseudogene']
duplicated_pgene_type = ['transcribed_unprocessed_pseudogene', 'translated_unprocessed_pseudogene', 'unprocessed_pseudogene']
unitary_pgene = ['transcribed_unitary_pseudogene', 'unitary_pseudogene']

In [6]:
processed_pgene = GRCh37[(GRCh37.Feature == 'gene') & ([j in processed_pgene_type for j in GRCh37.gene_type])]
duplicated_pgene = GRCh37[(GRCh37.Feature == 'gene') & ([j in duplicated_pgene_type for j in GRCh37.gene_type])]
unitary_pgene = GRCh37[(GRCh37.Feature == 'gene') & ([j in unitary_pgene for j in GRCh37.gene_type])]
protein_coding = GRCh37[(GRCh37.Feature == 'gene') & (GRCh37.gene_type == 'protein_coding')]

In [8]:
processed_pgene.to_bed('parent/fasta/processed_pgene.bed', chain = False)
duplicated_pgene.to_bed('parent/fasta/duplicated_pgene.bed', chain = False)
unitary_pgene.to_bed('parent/fasta/unitary_pgene.bed', chain = False)
protein_coding.to_bed('parent/fasta/protein_coding.bed', chain = False)

/Users/yzjiang/opt/miniconda3/lib/python3.9/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return pd.concat([outdf, df.get(noncanonical)], axis=1)
/Users/yzjiang/opt/miniconda3/lib/python3.9/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return pd.concat([outdf, df.get(noncanonical)], axis=1)
/Users/yzjiang/opt/miniconda3/lib/python3.9/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return pd.concat([outdf, df.get(noncanonical)], axis=1)
/Users/yzjiang/opt/miniconda3/lib/python3.9/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return pd.concat([outdf, df.get(noncanonical)], axis=1)
/Use

In [50]:
! gunzip fasta/GRCh37.primary_assembly.genome.fa.gz

In [10]:
! cat parent/fasta/duplicated_pgene.bed | awk -F '\t' '{print $1"\t"$2"\t"$3"\t"$10"\t"$5"\t"$6}' | bedtools getfasta -fi parent/fasta/GRCh37.primary_assembly.genome.fa -fo parent/fasta/duplicated_pgene.fa -bed - -nameOnly -s
! cat parent/fasta/processed_pgene.bed | awk -F '\t' '{print $1"\t"$2"\t"$3"\t"$10"\t"$5"\t"$6}' | bedtools getfasta -fi parent/fasta/GRCh37.primary_assembly.genome.fa -fo parent/fasta/processed_pgene.fa -bed - -nameOnly -s
! cat parent/fasta/unitary_pgene.bed | awk -F '\t' '{print $1"\t"$2"\t"$3"\t"$10"\t"$5"\t"$6}' | bedtools getfasta -fi parent/fasta/GRCh37.primary_assembly.genome.fa -fo parent/fasta/unitary_pgene.fa -bed - -nameOnly -s
! cat parent/fasta/protein_coding.bed | awk -F '\t' '{print $1"\t"$2"\t"$3"\t"$10"\t"$5"\t"$6}' | bedtools getfasta -fi parent/fasta/GRCh37.primary_assembly.genome.fa -fo parent/fasta/protein_coding.fa -bed - -nameOnly -s

In [2]:
rheMac10 = pr.read_gtf('GTF/rheMac10.gtf_polished')

In [3]:
rheMac10

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,matches_ref_protein,valid_ORF,missing_stop_codon,missing_start_codon,gene_status,remap_substituted_missing_target,transcript_status,ont,inframe_stop_codon,partial_ORF
0,chr1,Liftoff,gene,71528,80806,.,+,.,ENSG00000171163.16_11,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,Liftoff,transcript,71528,72824,.,+,.,ENSG00000171163.16_11,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,Liftoff,exon,71528,71658,.,+,.,ENSG00000171163.16_11,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,Liftoff,exon,72368,72824,.,+,.,ENSG00000171163.16_11,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,Liftoff,transcript,71556,73120,.,+,.,ENSG00000171163.16_11,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3067821,chrY,Liftoff,transcript,11193131,11199257,.,-,.,ENSG00000229709.1_5,unprocessed_pseudogene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGO:0000005,NaN,NaN
3067822,chrY,Liftoff,exon,11193131,11193261,.,-,.,ENSG00000229709.1_5,unprocessed_pseudogene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGO:0000005,NaN,NaN
3067823,chrY,Liftoff,exon,11194977,11195067,.,-,.,ENSG00000229709.1_5,unprocessed_pseudogene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGO:0000005,NaN,NaN
3067824,chrY,Liftoff,exon,11197046,11197220,.,-,.,ENSG00000229709.1_5,unprocessed_pseudogene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGO:0000005,NaN,NaN


# Macaque (rheMac10, Liftover from human GENCODE GTF Release 39)

In [41]:
rheMac10 = pr.read_gtf('GTF/rheMac10.gtf_polished')

## Macaque Pseudogene
10,935 pgenes (28,943 exons) -> 6,669 pgenes (20,536 exons) after removing exons overlapping with protein coding genes and lncRNAs -> 4,256 pgenes (13,750 exons) after selecting exons within unique genomic regions -> 3,226 pgenes (7,473 exons) after selecting duplicated and processed pgenes overlapping with PseudoPipe's output (fraction > 0.5) -> 3,174 pgenes (6,074 exons) after removing exons shorter than 100 bp.

In [42]:
## extract pgenes from the GTF file
pgene_type = ['processed_pseudogene', 'transcribed_processed_pseudogene', 'transcribed_unitary_pseudogene', 'transcribed_unprocessed_pseudogene',
              'translated_processed_pseudogene', 'translated_unprocessed_pseudogene', 'unitary_pseudogene', 'unprocessed_pseudogene']

In [43]:
pgene = rheMac10[(rheMac10.Feature == 'gene') & ([j in pgene_type for j in rheMac10.gene_type])]
pexon = rheMac10[(rheMac10.Feature == 'exon') & ([j in pgene_type for j in rheMac10.gene_type])]
protein_coding = rheMac10[(rheMac10.Feature == 'gene') & (rheMac10.gene_type == 'protein_coding')]
protein_coding_exon = rheMac10[(rheMac10.Feature == 'exon') & (rheMac10.gene_type == 'protein_coding')]
lncRNA = rheMac10[(rheMac10.Feature == 'gene') & (rheMac10.gene_type == 'lncRNA')]

In [44]:
print(len(set(pexon.gene_id)), len(pexon))

10935 28943


In [45]:
overlap_1 = pexon.count_overlaps(protein_coding, strandedness = False, overlap_col = "Count")
overlap_2 = pexon.count_overlaps(lncRNA, strandedness = False, overlap_col = "Count")
pexon = pexon[(overlap_1.Count == 0) & (overlap_2.Count == 0)]

In [46]:
print(len(set(pexon.gene_id)), len(pexon))

6669 20536


In [47]:
pexon.to_gtf('pgene/macaque/pexon_filtered_by_overlappings.gtf')

In [48]:
! gtf2bed < pgene/macaque/pexon_filtered_by_overlappings.gtf > pgene/macaque/pexon_filtered_by_overlappings.bed

In [49]:
! cat mappability/macaque/rheMac10.bedgraph | awk '{if($4>0.95) print $0}' | \
bedtools coverage -a pgene/macaque/pexon_filtered_by_overlappings.bed -b - | \
awk -F '\t' '{if($NF==1) print $0}' | awk -F '\t' 'NF-=4' > pgene/macaque/pexon_filtered_by_mappability.bed 

In [37]:
! awk '{print $1"\t"$7"\t"$8"\t"($2+1)"\t"$3"\t"$5"\t"$6"\t"$9"\t"(substr($0, index($0,$10)))}' pgene/macaque/pexon_filtered_by_mappability.bed > pgene/macaque/pexon_filtered_by_mappability.gtf
! sed -i '' 's/zero_length_insertion.*//g' pgene/macaque/pexon_filtered_by_mappability.gtf

In [38]:
! gtf2bed < pgene/macaque/pexon_filtered_by_mappability.gtf > pgene/macaque/pexon_filtered_by_mappability.bed

In [39]:
! cat pgene/macaque/pexon_filtered_by_mappability.bed | grep -v unitary |\
bedtools coverage -a - -b pgene/macaque/ppipe/Macaca_mulatta.Mmul_10.pgene.bed |\
awk -F '\t' '{if($NF>= 0.5) print $0}' | awk -F '\t' 'NF-=4' |\
awk '{print $1"\t"$7"\t"$8"\t"($2+1)"\t"$3"\t"$5"\t"$6"\t"$9"\t"(substr($0, index($0,$10)))}' > pgene/macaque/dup_pro.gtf

In [40]:
! cat pexon_filtered_by_mappability.bed | grep unitary |\
awk '{print $1"\t"$7"\t"$8"\t"($2+1)"\t"$3"\t"$5"\t"$6"\t"$9"\t"(substr($0, index($0,$10)))}' > pgene/macaque/unitary.gtf

In [41]:
! cat pgene/macaque/dup_pro.gtf pgene/macaque/unitary.gtf > pgene/macaque/pexon_filtered_by_ppipe.gtf

In [42]:
pexon = pr.read_gtf('pgene/macaque/pexon_filtered_by_ppipe.gtf')
print(len(set(pexon.gene_id)), len(pexon))

3226 7473


In [43]:
pexon = pexon[pexon.lengths() >= 100]
print(len(set(pexon.gene_id)), len(pexon))

3174 6074


In [44]:
pexon.drop_duplicate_positions()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_support_level,tag,havana_transcript,Parent,exon_number,exon_id,remap_original_location,ID,hgnc_id,ont
0,chr1,Liftoff,exon,195749,195958,.,+,.,ENSG00000242529.1_8,transcribed_processed_pseudogene,...,NA,"basic,Ensembl_canonical",OTTHUMT00000097131.2_2,ENST00000425108.1_2,1,ENSE00001538663.2_1,chr1:-:248721993-248722201,exon_143204,HGNC:45000,"PGO:0000004,PGO:0000019"
1,chr1,Liftoff,exon,2562018,2562141,.,+,.,ENSG00000227728.2_9,unprocessed_pseudogene,...,NA,"basic,Ensembl_canonical",OTTHUMT00000097600.1_4,ENST00000398739.2_4,7,ENSE00001733801.1_1,chr1:-:246513779-246513901,exon_142344,NaN,PGO:0000005
2,chr1,Liftoff,exon,4519849,4520158,.,+,.,ENSG00000227735.1_5,processed_pseudogene,...,NA,"pseudo_consens,basic,Ensembl_canonical",OTTHUMT00000097172.1_2,ENST00000415043.1_2,1,ENSE00001619889.1_1,chr1:-:244598391-244598687,exon_141746,HGNC:24416,PGO:0000004
3,chr1,Liftoff,exon,4720125,4720802,.,+,.,ENSG00000225401.2_5,processed_pseudogene,...,NA,"pseudo_consens,basic,Ensembl_canonical",OTTHUMT00000096696.2_2,ENST00000435390.2_2,1,ENSE00001703145.2_1,chr1:-:244394976-244395660,exon_141598,HGNC:33317,PGO:0000004
4,chr1,Liftoff,exon,6211396,6212783,.,+,.,ENSG00000270818.1_7,processed_pseudogene,...,NA,"pseudo_consens,basic,Ensembl_canonical",OTTHUMT00000468799.1_2,ENST00000603868.1_2,1,ENSE00003488617.1_1,chr1:-:242882066-242883475,exon_141058,NaN,PGO:0000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000,chrY,Liftoff,exon,8631048,8631190,.,-,.,ENSG00000236429.1_5,unprocessed_pseudogene,...,NA,"basic,Ensembl_canonical",OTTHUMT00000100005.1_2,ENST00000432046.1_2,1,ENSE00001799927.1_1,chrY:-:18741843-18741986,exon_1552708,HGNC:38781,PGO:0000005
5001,chrY,Liftoff,exon,9772891,9773872,.,-,.,ENSG00000232634.1_5,processed_pseudogene,...,NA,"pseudo_consens,basic,Ensembl_canonical",OTTHUMT00000088875.1_2,ENST00000436364.1_2,1,ENSE00001666493.1_1,chrY:-:21221106-21222089,exon_1553373,HGNC:37662,PGO:0000004
5002,chrY,Liftoff,exon,10121178,10121559,.,-,.,ENSG00000275280.1_7,processed_pseudogene,...,NA,"basic,Ensembl_canonical",OTTHUMT00000475783.1_2,ENST00000619838.1_2,1,ENSE00003751494.1_1,chrY:+:7656968-7657352,exon_1550873,NaN,PGO:0000004
5003,chrY,Liftoff,exon,10360439,10360860,.,-,.,ENSG00000230764.1_5,unprocessed_pseudogene,...,NA,"basic,Ensembl_canonical",OTTHUMT00000345307.1_2,ENST00000435196.1_2,1,ENSE00001709680.1_1,chr7:+:57185788-57186202,exon_596565,HGNC:42053,PGO:0000005


In [45]:
pexon.to_gtf('pgene/macaque/pexon_macaque.gtf')

## Macaque Protein-coding Genes
19,131 protein-coding genes (1,257,296 exons) -> 18,416 protein-coding genes (1,161,637 exons) after removing exons overlapping with pseudogenes and lncRNAs -> 17,700 protein-coding genes (1,086,750 exons) after selecting exons within unique genomic regions -> 17,450 protein-coding genes (715,248 exons) after removing exons shorter than 100 bp.

In [55]:
print(len(set(protein_coding_exon.gene_id)), len(protein_coding_exon))

19131 1257296


In [56]:
overlap_1 = protein_coding_exon.count_overlaps(pgene, strandedness = False, overlap_col = "Count")
overlap_2 = protein_coding_exon.count_overlaps(lncRNA, strandedness = False, overlap_col = "Count")
protein_coding_exon = protein_coding_exon[(overlap_1.Count == 0) & (overlap_2.Count == 0)]

In [57]:
print(len(set(protein_coding_exon.gene_id)), len(protein_coding_exon))

18416 1161637


In [58]:
protein_coding_exon.to_gtf('protein_coding/macaque/protein_coding_exon_filtered_by_overlappings.gtf')

In [59]:
! gtf2bed < protein_coding/macaque/protein_coding_exon_filtered_by_overlappings.gtf > protein_coding/macaque/protein_coding_exon_filtered_by_overlappings.bed

In [60]:
! cat mappability/macaque/rheMac10.bedgraph | awk '{if($4>0.95) print $0}' | \
bedtools coverage -a protein_coding/macaque/protein_coding_exon_filtered_by_overlappings.bed -b - | \
awk -F '\t' '{if($NF==1) print $0}' | awk -F '\t' 'NF-=4' > protein_coding/macaque/protein_coding_exon_filtered_by_mappability.bed 

In [61]:
! awk '{print $1"\t"$7"\t"$8"\t"($2+1)"\t"$3"\t"$5"\t"$6"\t"$9"\t"(substr($0, index($0,$10)))}' protein_coding/macaque/protein_coding_exon_filtered_by_mappability.bed  > protein_coding/macaque/protein_coding_exon_filtered_by_mappability.gtf
! sed -i '' 's/zero_length_insertion.*//g' protein_coding/macaque/protein_coding_exon_filtered_by_mappability.gtf

In [62]:
protein_coding_exon = pr.read_gtf('protein_coding/macaque/protein_coding_exon_filtered_by_mappability.gtf')
print(len(set(protein_coding_exon.gene_id)), len(protein_coding_exon))

17700 1086750


In [63]:
protein_coding_exon = protein_coding_exon[protein_coding_exon.lengths() >= 100]
print(len(set(protein_coding_exon.gene_id)), len(protein_coding_exon))

17450 715248


In [64]:
protein_coding_exon.to_gtf('protein_coding/macaque/protein_coding_macaque.gtf')

In [61]:
pexon_macaque = pr.read_gtf('pgene/macaque/pexon_macaque.gtf')
pexon_human = pr.read_gtf('pgene/human/pexon_human.gtf')

In [68]:
len(set(pexon_human.gene_id))

4067

In [66]:
len(set(pexon_macaque.gene_id))

3174

In [71]:
len(set(pexon_human.gene_id).intersection(set(pexon_macaque.gene_id)))

2488